In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import my_function
import time_dependent_tree
import random
from sklearn.model_selection import train_test_split
from statsmodels.tsa.seasonal import STL
import scipy.stats

from sklearn import tree
from sklearn.tree import DecisionTreeRegressor,plot_tree

In [2]:
# テストデータ設定部

np.random.seed(314) # 乱数シードを314に設定
# 合成波の作成
syn_t = my_function.make_wave(0.5, 1.0, 3.0, 100)[0]
syn_y = my_function.make_wave(0.5, 1.0, 3.0, 100)[1] + my_function.make_wave(0.75, 1.7, 3.0, 100)[1] + 0.05 * np.random.randn(len(syn_t))
syn_y = scipy.stats.zscore(syn_y)

In [3]:
# パラメータ等の設定
raw_array = syn_y
max_depth = 5

In [4]:
# データの整形
raw_x, raw_y = my_function.sliding_window(raw_array)
# データの分割
raw_x_train, raw_x_test, raw_y_train, raw_y_test = train_test_split(raw_x, raw_y, test_size=0.1, shuffle=False)

In [5]:
# モデル構築
clf = DecisionTreeRegressor(max_depth=max_depth)
clf.fit(X=raw_x_train,y=raw_y_train)

DecisionTreeRegressor(max_depth=5)

In [8]:
graph = graphviz.Source(tree.export_graphviz(clf, class_names=iris.feature_names, filled=True))
graph

NameError: name 'graphviz' is not defined